# Multiprocessing



In [1]:
import itertools
import sys
import os


sys.path.insert(0, "../02-sensitivity-analysis/python")

import multiprocessing as mp
import numpy as np
import matplotlib.pyplot as plt
from ishigami import *
from functools import partial

## Vectorization

In [2]:
seed, num_draws = 1, 100
np.random.seed(seed)
inputs = np.random.uniform(low=-np.pi, high=np.pi, size=(num_draws, 3))

def evaluate_ishigami_loop(inputs):
    for input_ in inputs:
        evaluate_ishigami_readable(input_)

%timeit evaluate_ishigami_loop(inputs)
%timeit evaluate_ishigami(inputs)

644 µs ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
19.1 µs ± 484 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Multiprocessing
We first want to get a sense how many CPU's we have available.

In [3]:
print(f"Number of cpu : {mp.cpu_count()}")

Number of cpu : 4


In [4]:
# show groups of processes working on total and main effects., mpi worker setup nicely done as well as task differs.

## mp.Process

* without communication

In [5]:
num_outer, num_inner = 10000, 1000

import multiprocessing

def task(num_outer, num_inner, which):
    
    print(f"... started on input parameter {which}")
    rslt = compute_simulation_main_effect(num_outer, num_inner, which)
    print(f"... finished input parameter {which}")

if __name__ == '__main__':
    jobs = []
    for which in range(3):
        p = mp.Process(target=task, args=(num_outer, num_inner, which))
        jobs.append(p)
        p.start()

... started on input parameter 0
... started on input parameter 1
... started on input parameter 2
... finished input parameter 2
... finished input parameter 0
... finished input parameter 1


* with communication

In [6]:
def task(num_outer, num_inner, which, qout):
    rslt = compute_simulation_main_effect(num_outer, num_inner, which)
    qout.put((which, rslt))

if __name__ == '__main__':
    
    processes = []
    qout = mp.Queue()
    
    for which in range(3):
        p = mp.Process(target=task, args=(num_outer, num_inner, which, qout))
        processes.append(p)
        
    for p in processes:
        p.start()

    for p in processes:
        p.join()
        
    unsorted_result = [qout.get() for p in processes]
    result = [t[1] for t in sorted(unsorted_result)] 
    print(result)

[0.31568004510111963, 0.45194103402865243, 0.0010100507352421427]


## mp.Pool

In [8]:
def task(num_outer, num_inner, which):
    
    print(f"... started on input parameter {which}")
    rslt = compute_simulation_main_effect(num_outer, num_inner, which)
    print(f"... finished input parameter {which}")
    
pool = mp.Pool(processes = 3)
task_partial = partial(task, num_outer, num_inner)
pool.map(task_partial, range(3))

pool.close()
pool.join()

... started on input parameter 0
... started on input parameter 1
... started on input parameter 2
... finished input parameter 2
... finished input parameter 0
... finished input parameter 1


In [33]:
def task_manager(num_outer, num_inner, task):
    label, which = task
    print(f"working on {label} effect for input {which}")
    if label == "main":
        return compute_simulation_main_effect(num_outer, num_inner, which)
    elif label == "total":
        return compute_simulation_total_effect(num_outer, num_inner, which)
    else:
        raise NotImplementedError

tasks = list(itertools.product(["main", "total"], range(3)))

task_manager_partial = partial(task_manager, num_outer, num_inner)

pool = mp.Pool(processes = 3)
rslt = pool.map(task_manager_partial, tasks)

pool.close()
pool.join()

working on main effect for input 0
working on main effect for input 1
working on main effect for input 2
working on total effect for input 0
working on total effect for input 1
working on total effect for input 2


In [34]:
rslt

[0.31568004510111963,
 0.45194103402865243,
 0.0010100507352421427,
 0.5560154110550138,
 0.4445671468004033,
 0.24627081586804012]

## References

* https://www.journaldev.com/15631/python-multiprocessing-example#python-multiprocessing-process-queue-and-locks

* https://pymotw.com/2/multiprocessing/basics.html